In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Flatten, Dense, Dropout, Layer, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
class Inception():
    
    def __init__(self, filter_dims):
        
        if not len(filter_dims) == 6:
            raise ValueError('`filter_dims` must have a length of 6.')
        
        self.filter_dims = filter_dims
    
    def __call__(self, x):
            
        x1 = Conv2D(self.filter_dims[0], (1, 1), padding='same', activation='relu')(x)

        x2 = Conv2D(self.filter_dims[2], (1, 1), padding='same', activation='relu')(x)
        x2 = Conv2D(self.filter_dims[1], (3, 3), padding='same', activation='relu')(x)

        x3 = Conv2D(self.filter_dims[4], (1, 1), padding='same', activation='relu')(x)
        x3 = Conv2D(self.filter_dims[3], (5, 5), padding='same', activation='relu')(x)

        x4 = MaxPooling2D(pool_size=(3, 3), padding='same')(x)
        x4 = Conv2D(self.filter_dims[5], (1, 1), padding='same', activation='relu')(x)

        x = Concatenate(axis=3)([x1, x2, x3, x4])

        return x

In [3]:
K.clear_session()

inputs = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(inputs)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception([64,96,128,16,32,32])(x)
x = Inception([128,128,196,32,96,64])(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception([192,96,208,16,48,64])(x)
x = Inception([160,112,224,24,64,64])(x)
x = Inception([128,128,256,24,64,64])(x)
x = Inception([112,144,288,32,64,64])(x)
x = Inception([256,160,320,32,128,128])(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception([256,160,320,32,128,128])(x)
x = Inception([384,192,384,48,128,128])(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(1000, activation='softmax')(x)

inception = Model(inputs=inputs, outputs=x)
inception.compile(optimizer='sgd', loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
inception.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 192)  110784      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin